# Primer Intento de clasificador



Como el titulo lo dice, primer intento de clasificador usando como referencia el laboratorio x .
Se considera para este caso el archivo movies_final_imdb el cual contiene:
    
    movieId+nota_imbd_aprox+duracion+categoria+actores
    
    
Nota : Falta agregar los directores y quizas las fechas para ve como cambia. 

In [2]:
## Librerias
import csv
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.utils import shuffle

from sklearn import tree
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn import cross_validation

Cargamos el archivo 

In [86]:
#Cargamos el Archivo.

movies = pd.read_csv("movies_final.csv", header = 0)
score = pd.read_csv("imbd.csv", header = 0)

movies_header = list(movies.columns.values)

movies = movies.as_matrix()



score = np.ravel(score.as_matrix())
movies = movies[0:4400]
score = score[0:4400]


#score = np.asarray(score, dtype="|S6")
#print("Movies:\n", movies[4451])
#print("Score:\n", score[4451])
#print(movies_header)




# KNN sin Cross-Validation



In [89]:
# Primer Clasificador : K-nearnest-neighbors

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(movies, score)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [95]:
#Predict

score_pred = knn.predict(movies)
accuracy_score(score, score_pred)

print(classification_report(score,score_pred))

             precision    recall  f1-score   support

          2       0.00      0.00      0.00        15
          3       0.19      0.10      0.13        39
          4       0.38      0.10      0.15       168
          5       0.38      0.33      0.35       458
          6       0.52      0.64      0.58      1475
          7       0.52      0.57      0.55      1487
          8       0.54      0.37      0.44       723
          9       1.00      0.03      0.06        35

avg / total       0.50      0.51      0.50      4400



# KNN con Cross Validation


In [113]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.33, random_state=37)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

# Hacemos 
print(classification_report(y_test, y_pred))

#Hacemos el cross validation
cross_validation.cross_val_score(knn, movies, score, cv=50)

             precision    recall  f1-score   support

          2       0.00      0.00      0.00         6
          3       0.00      0.00      0.00        14
          4       0.07      0.06      0.07        51
          5       0.20      0.26      0.22       136
          6       0.38      0.44      0.41       494
          7       0.40      0.37      0.39       517
          8       0.32      0.23      0.27       226
          9       0.00      0.00      0.00         8

avg / total       0.34      0.34      0.34      1452



array([ 0.35416667,  0.33333333,  0.27083333,  0.27083333,  0.35416667,
        0.27083333,  0.25      ,  0.3125    ,  0.41666667,  0.39583333,
        0.3125    ,  0.375     ,  0.33333333,  0.29166667,  0.29166667,
        0.38297872,  0.17021277,  0.31914894,  0.31914894,  0.27659574,
        0.31914894,  0.31914894,  0.36170213,  0.26086957,  0.32608696,
        0.2826087 ,  0.30434783,  0.36956522,  0.26086957,  0.43478261,
        0.32608696,  0.2173913 ,  0.41304348,  0.30434783,  0.34782609,
        0.4       ,  0.37777778,  0.33333333,  0.4       ,  0.25      ,
        0.31818182,  0.36363636,  0.43181818,  0.25      ,  0.47727273,
        0.38636364,  0.43181818,  0.38636364,  0.31818182,  0.45454545,
        0.22727273,  0.40909091,  0.47727273,  0.31818182,  0.36363636,
        0.36363636,  0.31818182,  0.36363636,  0.30232558,  0.37209302,
        0.39534884,  0.37209302,  0.34883721,  0.41860465,  0.39534884,
        0.41860465,  0.27906977,  0.41860465,  0.38095238,  0.30

Aparentemente no es buen clasificador para estos casos. ¿Por qué?

# Support Vector Machine - SVM

In [120]:

movies_svm = svm.SVC()
movies_svm.fit(movies, score)  

score_pred = knn.predict(movies)
accuracy_score(score, score_pred)

print(classification_report(score,score_pred))


             precision    recall  f1-score   support

          2       0.56      0.60      0.58        15
          3       0.57      0.82      0.67        39
          4       0.71      0.69      0.70       168
          5       0.68      0.72      0.70       458
          6       0.77      0.79      0.78      1475
          7       0.80      0.74      0.77      1487
          8       0.74      0.77      0.76       723
          9       0.75      0.77      0.76        35

avg / total       0.76      0.76      0.76      4400



# SVM con Cross-Validation

In [131]:
movies_svm = svm.SVC()
cross_validation.cross_val_score(movies_svm, movies, score, cv=2)

# cv = 3
#array([ 0.42029973,  0.39305177,  0.41598361])


#cv = 5
# array([ 0.40816327,  0.41156463,  0.39954597,  0.4214123 ,  0.41619156])


#cv = 2
# array([ 0.41125738,  0.41010469])


array([ 0.41125738,  0.41010469])

# SVM test set manual

In [127]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.5)
#, random_state=37

movies_svm = svm.SVC()
movies_svm.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)


0.76636363636363636

Misteriosamente al tomar un 33% y un 50% del dataset, el resultado es bastante bueno.